The goal here is to build a FSQ-VAE which builds latent vectors for spatio-temporal "tublets" described in the ViViT. Sequences of tublet latent vectors are to then be modeled by a Transformer Decoder

![](https://i.imgur.com/9G7QTfV.png)

In [ ]:
!pip install -q wandb pytorch_lightning av imageio

In [1]:
import os
import gc
import math
import wandb
import imageio
import torch
import torchvision
import numpy as np
import pytorch_lightning as pl
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML
from torch import nn, optim
from torch.nn import functional as F
from torchvision.datasets.video_utils import VideoClips
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import Compose, Lambda, Resize, ToTensor, CenterCrop, Grayscale
import torchvision.transforms.functional as TF

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, EarlyStopping
from pytorch_lightning.tuner import Tuner
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR

pl.seed_everything(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
steamboat_willie_gdrive_path = '/content/drive/My Drive/SteamboatWillie/SteamboatWillie.mp4'
!cp -r /content/drive/My\ Drive/SteamboatWillie/clips .

# Dataset

In [ ]:
class RandomHorizontalFlipVideo(torch.nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def forward(self, x):
        # x shape is expected to be (C, T, H, W)
        if torch.rand(1) < self.p:
            # Flip all frames in the clip
            return x.flip(-1)
        return x


class SteamboatWillieDataset(Dataset):
    def __init__(self, config, mode='train', train_split=0.8):
        super().__init__()
        self.config = config
        self.train_split = train_split
        self.mode = mode

        self.preprocess_transforms = Compose([
                Lambda(lambda x: x.permute(0, 3, 1, 2)),   # (T, H, W, C) to (T, C, H, W) for Greyscale
                Grayscale(num_output_channels=1),          # Convert to grayscale
                Lambda(lambda x: x.permute(1, 0, 2, 3)),   # (T, C, H, W) to (C, T, H, W) for Conv3d
                CenterCrop((480, 575)),                    # Center crop to remove virtical bars
                Resize((config.img_size, config.img_size))
        ])

        self.postprocess_transforms = Compose([
            Lambda(lambda x: x / 255.),
            Lambda(lambda x: x.view(self.config.in_channels, self.config.clip_length, self.config.img_size, self.config.img_size))
        ])

        if self.mode == 'train':
            self.postprocess_transforms.transforms.append(RandomHorizontalFlipVideo(p=0.5))

        if os.path.exists(config.dest_dir):
            clip_paths = self.build_existing_clip_paths(config.dest_dir)
            self.clips = self.build_clip_refs(clip_paths)
        else:
            video_clips = VideoClips(
                config.paths,
                clip_length_in_frames=config.clip_length,
                frames_between_clips=config.clip_length
            )

            self.clips = self.build_clip_refs(self.build_clip_paths(video_clips, self.preprocess_transforms, config.dest_dir))

        if mode in ['train', 'val']:
            total_clips = len(self.clips)

            indices = torch.randperm(total_clips).tolist()
            train_size = int(total_clips * train_split)

            if mode == 'train':
                self.clip_indices = indices[:train_size]
            else:
                self.clip_indices = indices[train_size:]
        else:
            self.clip_indices = list(range(len(self.clips)))

    def build_clip_paths(self, video_clips, transforms, dest_dir):
        """
        Build set of binary files to store processed video clips
        returns dict of clip_idx -> mmapped file path
        """
        clip_paths = {}

        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)

        for idx in tqdm(range(video_clips.num_clips()), desc='Creating clip .bin files'):
            # transform clips and write to mmap file
            clip, _, _, _ = video_clips.get_clip(idx)
            clip = self.preprocess_transforms(clip)
            clip_np = clip.numpy().astype(np.uint8)

            mmapped_file_path = os.path.join(dest_dir, f'clip_{idx}.bin')
            fp = np.memmap(mmapped_file_path, dtype='uint8', mode='w+', shape=clip_np.shape)
            fp[:] = clip_np[:]
            fp.flush()
            del fp
            clip_paths[idx] = mmapped_file_path

        return clip_paths

    def build_existing_clip_paths(self, dest_dir):
        """"
        returns dict of clip_idx -> mmapped file path
        from existing .bin files
        """
        clips_paths = {}
        for filename in os.listdir(dest_dir):
            if filename.startswith('clip_') and filename.endswith('.bin'):
                idx = int(filename.split('_')[1].split('.')[0])
                file_path = os.path.join(dest_dir, filename)
                clips_paths[idx] = file_path

        return clips_paths

    def build_clip_refs(self, clip_paths):
        """
        Build mmap reference to bin files
        returns dict of clip_idx -> np.array mmapped to respective bin file
        """
        clips = {}
        for idx, path in tqdm(clip_paths.items(), desc='Building clip refs'):
            clips[idx] = np.memmap(path, dtype='uint8', mode='r')

        return clips

    def __len__(self):
        return len(self.clip_indices)

    def __getitem__(self, idx):
        clip = self.clips[self.clip_indices[idx]]
        return self.postprocess_transforms(torch.tensor(clip, dtype=torch.float32))


class SteamboatWillieDataModule(pl.LightningDataModule):
    def __init__(self, config):
        super().__init__()
        self.batch_size = config.batch_size
        self.config = config

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train_dataset = SteamboatWillieDataset(self.config, mode='train')
            self.val_dataset = SteamboatWillieDataset(self.config, mode='val')

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.config.num_workers)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.config.num_workers)

# Model

In [ ]:
class Encoder(nn.Module):
    def __init__(
            self,
            in_channels,
            hidden_channels,
            out_channels,
            nlayers=3
        ):
        super().__init__()
        self.downsample_blocks = nn.Sequential(*[
            nn.Sequential(
                nn.Conv3d(
                    in_channels=in_channels if i==0 else hidden_channels,
                    out_channels=out_channels if i==nlayers-1 else hidden_channels,
                    kernel_size=(3, 3, 3),
                    stride=(2, 2, 2),
                    padding=(1, 1, 1)
                ),
                nn.BatchNorm3d(out_channels if i==nlayers-1 else hidden_channels),
                nn.ReLU()
            ) for i in range(nlayers)
        ])

    def forward(self, x):
        # ((B * n_t * n_h * n_w), C, t, p, p)
        x = self.downsample_blocks(x)
        # ((B * n_t * n_h * n_w), C, 1, 2, 2) -> ((B * n_t * n_h * n_w), (C * 1 * 2 * 2))
        x = torch.flatten(x, start_dim=1)
        return x

In [ ]:
class Decoder(torch.nn.Module):
    def __init__(
            self,
            in_channels,
            hidden_channels,
            out_channels,
            nlayers=3,
        ):
        super().__init__()

        self.upsample_blocks = nn.Sequential(*[
            nn.Sequential(
                nn.ConvTranspose3d(
                    in_channels=in_channels if i==0 else hidden_channels,
                    out_channels=out_channels if i==nlayers-1 else hidden_channels,
                    kernel_size=(3, 3, 3),
                    stride=(2, 2, 2),
                    padding=(1, 1, 1),
                    output_padding=(1, 1, 1)
                    ),
                nn.BatchNorm3d(out_channels if i==nlayers-1 else hidden_channels) if i<nlayers-1 else nn.Identity(),
                nn.ReLU() if i<nlayers-1 else nn.Identity()
            ) for i in range(nlayers)
        ])

        self.out_act = nn.Sigmoid()

    def forward(self, x):
        # ((B * n_t * n_h * n_w), (C * 1 * 2 * 2)) -> ((B * n_t * n_h * n_w), C, 1, 2, 2)
        x = x.reshape(x.shape[0], -1, 1, 2, 2)
        # ((B * n_t * n_h * n_w), C, 1, 2, 2)
        x = self.upsample_blocks(x)
        x = self.out_act(x)
        return x

In [ ]:
class FSQ(nn.Module):
    def __init__(self, levels, eps=1e-3):
        super().__init__()
        self.register_buffer('levels', torch.tensor(levels))
        self.register_buffer(
            'basis',
            torch.cumprod(torch.tensor([1] + levels[:-1]), dim=0, dtype=torch.int32)
        )

        self.eps = eps
        self.codebook_size = torch.prod(self.levels)

        # self.register_buffer('implicit_codebook', self.idxs_to_code(torch.arange(self.codebook_size)))

    def round_ste(self, z):
        z_q = torch.round(z)
        return z + (z_q - z).detach()

    def quantize(self, z):
        # half_l is used to determine how to scale tanh; we
        # subtract 1 from the number of levels to account for 0
        # being a quantization bin and tanh being symmetric around 0
        half_l = (self.levels - 1) * (1 - self.eps) / 2

        # if a given level is even, it will result in a scale for tanh
        # which is halfway between integer values, so we offset
        # the tanh output down by 0.5 to line it with whole integers
        offset = torch.where(self.levels % 2 == 0, 0.5, 0.0)

        # if our level is even, we want to shift the tanh input to
        # ensure the 0 quantization bin is centered
        shift = torch.tan(offset / half_l)

        # once we have our shift and offset (in the case of an even level)
        # we can round to the nearest integer bin and allow for STE
        z_q = self.round_ste(torch.tanh(z + shift) * half_l - offset)

        # after quantization, we want to renormalize the quantized
        # values to be within the range expected by the model (ie. [-1, 1])
        half_width = self.levels // 2
        return z_q / half_width

    def scale_and_shift(self, z_q_normalized):
        half_width = self.levels // 2
        return (z_q_normalized * half_width) + half_width

    def scale_and_shift_inverse(self, z_q):
        half_width = self.levels // 2
        return (z_q - half_width) / half_width

    def code_to_idxs(self, z_q):
        z_q = self.scale_and_shift(z_q)
        return (z_q * self.basis).sum(dim=-1).to(torch.int32)

    def idxs_to_code(self, idxs):
        idxs = idxs.unsqueeze(-1)
        codes_not_centered = (idxs // self.basis) % self.levels
        return self.scale_and_shift_inverse(codes_not_centered)

    def forward(self, z):
        # TODO: make this work for generic tensor sizes
        # TODO: use einops to clean up
        
        # B, C, T, H, W = z.shape

        # # (B, C, T, H, W) -> (B, T, H, W, C)
        # z_c_last = z.permute(0, 2, 3, 4, 1).contiguous()

        # # (B, T, H, W, C) -> (BTHW, C)
        # z_flatten = z_c_last.reshape(-1, C)

        # z_flatten_q = self.quantize(z_flatten)

        # # (BTHW, C) -> (B, T, H, W, C) -> (B, C, T, H, W)
        # z_q = z_flatten_q.reshape(B, T, H, W, C).permute(0, 4, 1, 2, 3).contiguous()
        
        # z already of shape (B, C)
        z_q = self.quantize(z)

        return {'z_q': z_q}

In [ ]:
class TubeletFSQVAE(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = Encoder(
            in_channels     = config.in_channels,
            hidden_channels = config.hidden_channels,
            out_channels    = config.latent_channels,
        )
        self.decoder = Decoder(
            in_channels     = config.latent_channels,
            hidden_channels = config.hidden_channels,
            out_channels    = config.in_channels
        )

        self.quantizer = FSQ(config.levels)

        self.config = config

    def encode(self, inputs):
        inputs = self.extract_tubelets(inputs)
        return self.encoder(inputs)

    def quantize(self, z):
        return self.quantizer(z)

    def decode(self, z_q):
        return self.decoder(z_q)

    def extract_tubelets(self, x, t=8, p=16):
        '''
        extract tubelet sequence of shape ((B * n_t * n_h * n_w), C, t, p, p)
        from a video tensor of shape (B, C, T, H, W)
        where n_t = T // t, n_h = H // p, and n_w = W // p
        '''
        assert len(x.shape) == 5, 'vid.shape must be (B, C, T, H, W)'

        B, C, T, H, W = x.shape

        assert T % t == 0, 't must divide T (vid.shape[2]) evenly'
        assert H % p == 0, 'p must divide H (vid.shape[3]) evenly'
        assert W % p == 0, 'p must divide W (vid.shape[4]) evenly'

        n_t, n_h, n_w = T // t, H // p, W // p

        # (B, C, T, H, W) -> (B, C, n_t, t, n_h, p, n_w, p)
        x = x.reshape(B, C, n_t, t, n_h, p, n_w, p)
        # (B, C, n_t, t, n_h, p, n_w, p) -> (B, n_t, n_h, n_w, C, t, p, p)
        x = x.permute(0, 2, 4, 6, 1, 3, 5, 7).contiguous()
        # (B, n_t, n_h, n_w, C, t, p, p) -> ((B * n_t * n_h * n_w), C, t, p, p)
        x = x.reshape(-1, C, t, p, p)

        return x

    def loss(self, x_hat, x, quantized):
        MSE = F.mse_loss(x_hat, x)

        return {
            'loss': MSE,
            **quantized
        }

    def forward(self, x):
        z = self.encode(x)
        quantized = self.quantize(z)
        x_hat = self.decode(quantized['z_q'])
        losses = self.loss(x_hat, self.extract_tubelets(x), quantized)

        return {'x_hat': x_hat, **losses}

In [ ]:
def extract_tubelets(x, t=8, p=16):
    '''
    extract tubelet sequence of shape ((B * n_t * n_h * n_w), C, t, p, p)
    from a video tensor of shape (B, C, T, H, W)
    where n_t = T // t, n_h = H // p, and n_w = W // p
    '''
    assert len(x.shape) == 5, 'vid.shape must be (B, C, T, H, W)'

    B, C, T, H, W = x.shape

    assert T % t == 0, 't must divide T (vid.shape[2]) evenly'
    assert H % p == 0, 'p must divide H (vid.shape[3]) evenly'
    assert W % p == 0, 'p must divide W (vid.shape[4]) evenly'

    n_t, n_h, n_w = T // t, H // p, W // p

    # (B, C, T, H, W) -> (B, C, n_t, t, n_h, p, n_w, p)
    x = x.reshape(B, C, n_t, t, n_h, p, n_w, p)
    # (B, C, n_t, t, n_h, p, n_w, p) -> (B, n_t, n_h, n_w, C, t, p, p)
    x = x.permute(0, 2, 4, 6, 1, 3, 5, 7).contiguous()
    # (B, n_t, n_h, n_w, C, t, p, p) -> ((B * n_t * n_h * n_w), C, t, p, p)
    x = x.reshape(-1, C, t, p, p)

    return x

# Lightning Module

In [ ]:
class TubeletFSQVAE(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = Encoder(
            in_channels     = config.in_channels,
            hidden_channels = config.hidden_channels,
            out_channels    = config.latent_channels,
        )
        self.decoder = Decoder(
            in_channels     = config.latent_channels,
            hidden_channels = config.hidden_channels,
            out_channels    = config.in_channels
        )

        self.quantizer = FSQ(config.levels)

        self.config = config

    def encode(self, inputs):
        inputs = self.extract_tubelets(inputs)
        return self.encoder(inputs)

    def quantize(self, z):
        return self.quantizer(z)

    def decode(self, z_q):
        return self.decoder(z_q)

    def extract_tubelets(self, x, t=8, p=16):
        '''
        extract tubelet sequence of shape ((B * n_t * n_h * n_w), C, t, p, p)
        from a video tensor of shape (B, C, T, H, W)
        where n_t = T // t, n_h = H // p, and n_w = W // p
        '''
        assert len(x.shape) == 5, 'vid.shape must be (B, C, T, H, W)'

        B, C, T, H, W = x.shape

        assert T % t == 0, 't must divide T (vid.shape[2]) evenly'
        assert H % p == 0, 'p must divide H (vid.shape[3]) evenly'
        assert W % p == 0, 'p must divide W (vid.shape[4]) evenly'

        n_t, n_h, n_w = T // t, H // p, W // p

        # (B, C, T, H, W) -> (B, C, n_t, t, n_h, p, n_w, p)
        x = x.reshape(B, C, n_t, t, n_h, p, n_w, p)
        # (B, C, n_t, t, n_h, p, n_w, p) -> (B, n_t, n_h, n_w, C, t, p, p)
        x = x.permute(0, 2, 4, 6, 1, 3, 5, 7).contiguous()
        # (B, n_t, n_h, n_w, C, t, p, p) -> ((B * n_t * n_h * n_w), C, t, p, p)
        x = x.reshape(-1, C, t, p, p)

        return x

    def loss(self, x_hat, x, quantized):
        MSE = F.mse_loss(x_hat, x)

        return {
            'loss': MSE,
            **quantized
        }

    def forward(self, x):
        z = self.encode(x)
        quantized = self.quantize(z)
        x_hat = self.decode(quantized['z_q'])
        losses = self.loss(x_hat, self.extract_tubelets(x), quantized)

        return {'x_hat': x_hat, **losses}

# Config

In [ ]:
class VideoVAEConfig:
    def __init__(self):
        self.project_name = 'Tubelet FSQ-VAE Steamboat Willie'
        # dataset properties
        self.paths = ['/content/drive/My Drive/SteamboatWillie/SteamboatWillie.mp4']
        self.dest_dir = './clips/'
        # model checkpoints
        self.checkpoint_path = "./checkpoints"
        self.save_top_k = 1
        # training
        self.train_split = 0.8
        self.batch_size = 16
        self.max_epochs = 500
        self.training_steps = 100000
        self.num_workers = 2
        # optimizer
        self.lr = 5e-2
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.weight_decay = 0.0 # 1e-2
        self.use_wd_schedule = False
        self.use_lr_schedule = True
        # input properties
        self.clip_length = 16
        self.img_size = 256
        self.in_channels = 1
        # quantization
        self.quant_mode = 'fsq' # 'vq'
        self.latent_channels = 128 # 8
        self.codebook_size = 512
        self.commit_loss_beta = 0.25
        self.track_codebook = True
        self.use_ema = True
        self.ema_gamma = 0.99
        self.level = 7
        self.levels = [self.level for _ in range(self.latent_channels * 4)]
        # encoder/decoder
        self.hidden_channels = 256
        self.start_channels = 32
        self.nblocks = 5
        self.nlayers = 3

    def update(self, updates):
        for key, value in updates.items():
            if hasattr(self, key):
                setattr(self, key, value)

    def to_dict(self):
        return {k: v for k, v in self.__dict__.items()}

# Component Testing

In [ ]:
config = VideoVAEConfig()
data_module = SteamboatWillieDataModule(config)
data_module.prepare_data()
data_module.setup()

train_loader = data_module.train_dataloader()
train_batch = next(iter(train_loader))
print(f'train_batch.shape: {train_batch.shape}')

In [ ]:
tublets = extract_tubelets(train_batch)
print(tublets.shape)

In [ ]:
enc=Encoder(in_channels=1,
            hidden_channels=32,
            out_channels=5)
fsq=FSQ(levels=[5, 5, 5, 5, 5])
dec=Decoder(in_channels=5,
            hidden_channels=32,
            out_channels=1)

In [ ]:
tubelets_enc = enc(tublets)
print(tubelets_enc.shape)

In [ ]:
tubelets_fsq = fsq(tubelets_enc)
print(tubelets_fsq['z_q'].shape)

In [ ]:
tubelets_dec = dec(tubelets_fsq['z_q'])
print(tubelets_dec.shape)

# Display Clips

In [ ]:
def display_clip(clip):
    def update(frame_idx):
        ax.clear()
        ax.imshow(video_clip_np[frame_idx], cmap='gray')
        ax.axis('off')

    video_clip_np = clip.permute(1, 2, 3, 0).numpy()
    fig, ax = plt.subplots()
    ani = FuncAnimation(fig, update, frames=range(video_clip_np.shape[0]), interval=50)
    plt.close()
    display(HTML(ani.to_html5_video()))

In [ ]:
tublets = extract_tubelets(train_batch)
print(tublets.shape)

In [ ]:
display_clip(tublets[1001])

# Initial Tubelet Shape Exploration

In [ ]:
# (B, C, T, H, W) -> (B, C, n_t, t, n_h, p, n_w, p)
tubelets = vid.reshape(B, C, n_t, t, n_h, p, n_w, p)
print(tubelets.shape)

In [ ]:
# (B, C, n_t, t, n_h, p, n_w, p) -> (B, n_t, n_h, n_w, C, t, p, p)
tubelets = tubelets.permute(0, 2, 4, 6, 1, 3, 5, 7).contiguous()
print(tubelets.shape)

In [ ]:
# (B, n_t, n_h, n_w, C, t, p, p) -> ((B * n_t * n_h * n_w), C, t, p, p)
tubelets = tubelets.reshape(-1, C, t, p, p)
print(tubelets.shape)

In [ ]:
conv3d_1 = nn.Conv3d(in_channels=C, out_channels=32, kernel_size=3, stride=2, padding=1)
out1 = conv3d_1(tubelets)
print(out1.shape)

In [ ]:
conv3d_2 = nn.Conv3d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
out2 = conv3d_2(out1)
print(out2.shape)

In [ ]:
conv3d_3 = nn.Conv3d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
out3 = conv3d_3(out2)
print(out3.shape)

In [ ]:
conv3d_4 = nn.Conv3d(in_channels=32, out_channels=32, kernel_size=3, stride=(1, 2, 2), padding=1)
out4 = conv3d_4(out3)
print(out4.shape)

In [ ]:
out5 = out4.reshape(-1, 32)
print(out5.shape)

In [ ]:
# separate out batch dimension post VAE encoding since dim=0 is of shape (B * n_t * n_h * n_w)
out6 = out5.reshape(out5.shape[0] // (n_t * n_h * n_w), -1, 32)
print(out6.shape)

In [ ]:
out7 = out6.reshape(-1, 32)
print(out7.shape)

In [ ]:
out7 = out7.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
print(out7.shape)

In [ ]:
decoder1 = nn.ConvTranspose3d(in_channels=32, out_channels=32, kernel_size=3, stride=(1, 2, 2), padding=1, output_padding=(0, 1, 1))
out8 = decoder1(out7)
print(out8.shape)

In [ ]:
decoder2 = nn.ConvTranspose3d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1, output_padding=(1, 1, 1))
out9 = decoder2(out8)
print(out9.shape)

In [ ]:
decoder3 = nn.ConvTranspose3d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1, output_padding=(1, 1, 1))
out10 = decoder2(out9)
print(out10.shape)

In [ ]:
decoder3 = nn.ConvTranspose3d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1, output_padding=(1, 1, 1))
out11 = decoder2(out10)
print(out11.shape)